<br>
#  **Crwaling Data**
1. 데이터 크롤링 1번으로 부하 줄이기
1. 수집 가능한 format 모두 정리하기

전체계획
1. KRX 사이트에서 Market 데이터 수집하기
1. 신규기업 목록 확인하여, 새로운 기업이 있으면 Google에서 코드명 수집하여 이어 붙이기
1. 기업 Code들은 외래키를 사용하여 Yahoo/ Google 코드 모두 저장 (ERD)

<br>
## **1. GoogleFinance.Get**
googlefinance.get

<br>
### **01 시간대별 정보 수집**
시간대별 가격정보 수집하기

In [1]:
code = "KOSDAQ:091990"
from googlefinance.get import get_data
get_data(code, '3d')

,code,open,high,low,close,volume
date,,,,,,
2018-04-11,KOSDAQ:091990,103100.0,103200.0,100000.0,100100.0,751870
2018-04-12,KOSDAQ:091990,100500.0,101300.0,98300.0,98500.0,851185


In [2]:
df = get_data([code, 'KRX:005930', 'KRX:000660'], '3Y', '3600')
print(set(df.code))
df.iloc[:,[0,3,4]].head(2)

{'KOSDAQ:091990', 'KRX:005930'}


,code,low,close
2017-07-28 10:00:00,KOSDAQ:091990,43650.0,47450.0
2017-07-28 11:00:00,KOSDAQ:091990,47150.0,47700.0


In [3]:
get_data(code, '7M', '600').iloc[:,3:].head(2)

,low,close,volume
2017-09-18 09:10:00,49100.0,49250.0,135140
2017-09-18 09:20:00,48800.0,48900.0,60080


<br>
## **2. KRX 사이트를 활용하여 전종목 가격수집**
http://marketdata.krx.co.kr/main/main.jsp

<br>
### **01 시간대별 요약정보 수집**
시간대별 가격정보 수집하기

In [4]:
from data.finance import get_data_price
df = get_data_price()
df.head(2)

,종목코드,종목명,현재가,대비,등락률,거래량,거래대금,시가,고가,저가,시가총액,시가총액비중(%),상장주식수(천주),외국인 보유주식수,외국인 지분율(%)
0,005930,삼성전자,2443000,-1000,-0.0,201022,495246434000,2495000,2495000,2430000,313648204842000,16.34,128386494,NaN,NaN
1,000660,SK하이닉스,81200,800,1.0,3053453,249613086540,82400,82400,81000,59113792038000,3.08,728002365,NaN,NaN


In [5]:
from datetime import datetime
hour = str(datetime.today().hour)
if len(hour) < 2: hour = '0' + hour

df_min         = df.iloc[:,[0,4,5]]
df_min.columns = ['Code', 'C'+hour, 'Volume'+hour]
df_min.to_csv('./'+hour+'.csv', encoding='ms949')
df_min.head(2)

,Code,C20,Volume20
0,005930,-0.0,201022
1,000660,1.0,3053453


<br>
### **02 기업 세부정보 수집**
기업정보 수집하기

In [6]:
from data.finance import get_data_info
df_info = get_data_info()
df_info.head(2)

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,BYC,001460,봉제의복 제조업,"메리야스,란제리 제조,도매/건축공사/부동산 임대,분양,공급",1975-06-02,12월,유 중 화,http://www.byc.co.kr,서울특별시
1,CJ CGV,079160,"영화, 비디오물, 방송프로그램 제작 및 배급업","영화상영,영화관 운영",2004-12-24,12월,서 정,http://www.cgv.co.kr,서울특별시


<br>
### **03 Kospi & Kosdaq 기호 붙이기**
상장사와 코스닥 구분 후 기호 붙이기

In [7]:
# {dict} 객체 삭제하기
a = {'a' : 1, 'b' : 2, 'c' : 3}
# a.pop('a', None)
# del a['a']   # 1개씩 제외만 된다

In [8]:
from data.finance import get_data_codes
code_df = get_data_codes()
code_df.head(3)

,종목코드,종목명,google,yahoo
0,005930,삼성전자,KRX:005930,005930.KS
1,000660,SK하이닉스,KRX:000660,000660.KS
2,207940,삼성바이오로직스,KRX:207940,207940.KS


In [9]:
code_df.shape

(2314, 4)

<br>
## **2. Dart 재무제표 수집**
https://datascienceschool.net/view-notebook/adead36729704e7b8660dda3be6a6524/
1. 금융감독원 인증키(Dart.kss.go.kr)
1. 인증키 : 8e30368d048361ba865f916c30739160a0e58ddd
1. 발급일 : 2017-08-03
1. 제한 횟수 : 10,000
1. 가능한 기능
    1. 기업 개황 제공
    1. 공시 정보 검색

<br>
### ** 01 기업개황 데이터 수집**
응답결과는 JSON 파일 형식으로 각 필드의 의를 한글로 변경

In [3]:
from data.finance import get_dart
df = get_dart('005930')
df.head(8)

,설명,내용
0,에러코드,000
1,에러메시지,정상
2,정식명칭,삼성전자(주)
3,영문명칭,"SAMSUNG ELECTRONICS CO,.LTD"
4,약식명칭,삼성전자
5,종목코드,005930
6,대표자명,"김기남, 김현석, 고동진"
7,법인구분,Y


<br>
### ** 02 공시정보 수집**
여기에서 auth 값은 개인의 인증키 문자열이다. 이 뒤에 다음과 같은 파라미터를 추가하여야 한다.
1. bsn_tp : 정기공시(5), 주요사항보고(3), 발행공시(11), 지분공시(4), 기타공시(9), 외부감사관련(3), 펀드공시(3), 자산유동화(6), 거래소공시(6), 공정위공시(5) (상세 유형 참조)
1. callback : 콜백함수명(JSONP용)
1. crp_cd : 공시대상회사의 종목코드(상장사:숫자 6자리) 또는 고유번호(기타법인:숫자 8자리)
1. dsp_tp : 정기공시(A), 주요사항보고(B), 발행공시(C), 지분공시(D), 기타공시(E), 외부감사관련(F), 펀드공시(G), 자산유동화(H), 거래소공시(I), 공정위공시(J)
1. end_dt : 검색종료 접수일자(YYYYMMDD) : 없으면 당일
1. fin_rpt : 최종보고서만 검색여부(Y or N) 기본값 : N (정정이 있는 경우 최종정정만 검색)
1. series : 오름차순(asc), 내림차순(desc) 기본값 : desc
1. start_dt : 검색시작 접수일자(YYYYMMDD) : 없으면 end_dt crp_cd가 없는 경우 검색기간은 3개월로 제한
1. sort : 접수일자(date), 회사명(crp), 보고서명(rpt) 기본값 : date
1. page_no : 페이지 번호(1~n) 기본값 : 1
1. page_set : 페이지당 건수(1~100) 기본값 : 10, 최대값 : 100
1. err : 에러코드(오류 메시지 참조)
1. err_msg : 에러메시지(오류 메시지 참조)
1. page_no : 페이지 번호
1. page_set : 페이지당 건수
1. total_count : 총 건수
1. total_page : 총 페이지 수

In [1]:
from data.finance import get_dart_search
get_dart_search('005930', '20180101', '3')

page_no : 3  total_count : 61  total_page : 7


,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,005930,Y,삼성전자,삼성전자,20180223,20180223000494,,[기재정정]참고서류
1,005930,Y,삼성전자,삼성전자,20180223,20180223000490,,[기재정정]주주총회소집공고
2,005930,Y,삼성전자,삼성전자,20180223,20180223800886,유,주주총회집중일개최사유신고
3,005930,Y,삼성전자,삼성전자,20180223,20180223800808,유,기타경영사항(자율공시)
4,005930,Y,삼성전자,삼성전자,20180223,20180223000314,정,참고서류
5,005930,Y,삼성전자,삼성전자,20180223,20180223000294,정,주주총회소집공고
6,005930,Y,삼성전자,삼성전자,20180223,20180223800573,유,주주총회소집결의
7,005930,Y,삼성전자,삼성전자,20180214,20180214000563,공,동일인등출자계열회사와의상품ㆍ용역거래변경
8,005930,Y,삼성전자,삼성전자,20180214,20180214000558,공,동일인등출자계열회사와의상품ㆍ용역거래변경
9,005930,Y,삼성전자,삼성전자,20180214,20180214800785,유,최대주주등소유주식변동신고서


In [14]:
def get_dart_search(code, date):
    import requests, json
    import pandas as pd
    url = "http://dart.fss.or.kr/api/search.json?auth={0}&crp_cd={1}&start_dt=" + date
    url = url.format("8e30368d048361ba865f916c30739160a0e58ddd", code)
    response = requests.get(url)
    response = json.loads(response.content)
    return response # pd.DataFrame(response['list'])


In [15]:
get_dart_search('005930', '20180301')

IndexError: tuple index out of range

In [5]:
date       = "20180301"
url_search = "http://dart.fss.or.kr/api/search.json?auth={0}&crp_cd={1}&start_dt=" + date
code       = "005930"
data       = response_json(url_search, code)
print('url info >> \nkeys : {}\ntotal page num :{}'.format(data.keys(), data['page_set']))

# 해당일 공개된 공시들 목록
# 1 page 당 8개를 출력
for i in data['list']:
    print(i["rpt_nm"])

NameError: name 'response_json' is not defined